In [1]:
import pandas as pd
from datetime import datetime
from cl_cleaning import CleaningText as ct
from ica_cierres import CierresF11
import numpy as np
# Configuraciones
pd.set_option('float_format', '{:,.2f}'.format)

In [2]:
pd.set_option('display.max_columns', 200)

In [4]:
res = pd.read_excel('output/cierres_f11/cd/210916-1619-cf11_cd_20-output.xlsx', sheet_name='210916-1619_cf11_cd_20', dtype='object')

In [5]:
res.columns

Index(['indice_cf11', 'nfolio', 'f12', 'prd_upc', 'qproducto', 'xobservacion',
       'total_costo_promedio', 'estado_actual', 'status_nuevo',
       'transportadora_nuevo', 'f3nuevo', 'f4_nuevo', 'reporte_a_contabilidad',
       'movimiento_contable', 'nc', 'tranf_electro_factura', 'pv', 'f5',
       'gco_dup', 'GCO', 'Comentario GCO', 'dupmc', 'checked', 'error_ru',
       'gco_dupall', 'fnandup'],
      dtype='object')

In [6]:
res['Comentario GCO'].unique()

array(['Coincidencia exacta F4+UPC+QTY',
       'Coincidencia F4+UPC+QTY + Registro duplicado en DB',
       'El tipo de F4 es diferente a dado de baja',
       'Cantidad de los F11s de un F4 > cantidad del F4',
       'Registro duplicado F12 + UPC + Cantidad', nan, 'Registro anulado',
       'Coincidencia F4+UPC+QTY - Se requiere F12|UPC|QTY para validar duplicidad',
       'Coincidencia exacta (F12|F11)', 'No coincide F4|UPC|QTY',
       'Coincidencia exacta F12',
       'Registro con año de creación diferente a 2021',
       'Coincidencia exacta F3+UPC+QTY', 'No coincide F5|UPC|Qty',
       'No coincide F12',
       'Coincidencia (F12|F11) + Registro duplicado en DB',
       'Registro sin nro. de F4',
       'Coincidencia F12 + Registro duplicado en DB',
       'Coincidencia F3+UPC+QTY + Registro duplicado en DB',
       'No coincide (F12|F11)', 'No coincide F3|UPC|QTY',
       'Coincidencia (F12|F11) - Se requiere F12|UPC|QTY para validar duplicidad',
       'Recibido con fecha ant

In [7]:
res.status_nuevo.unique()

array(['f4 de merma', 'f4 de merma - por duplicidad f12 + upc + cantidad',
       'registro duplicado en base de datos',
       'cierre x f4 cobrado a terceros',
       'cierre x producto guardado despues de inventario',
       'cierre x recupero con cliente - refacturacion - base fal.com',
       'cierre x f3 devuelto a proveedor', 'producto en tienda',
       'f4 de merma - no se puede generar producto no existe',
       'cierre x producto guardado despues de inventario - no aplica f12'],
      dtype=object)

In [8]:
res.groupby('GCO')['total_costo_promedio'].sum()

GCO
ANU       9069562
DUP      12370760
NAA      15438360
NCC        239770
NDB     126251673
NF4         64744
NFD      73148634
OKD     388638653
OKK    9892915340
Name: total_costo_promedio, dtype: int64

## Validación 4 

In [6]:
from ica_raw import InternalControlAnalysis

index_name = 'indice_cf11'
cost_column = 'total_costo_promedio'
status_column = 'status_nuevo'
qty_column = 'qproducto'
upc_column = 'prd_upc'

res.qproducto  = res.qproducto.astype('float')
ica = InternalControlAnalysis(res, index_name)

fcols = ['f3nuevo','f4_nuevo','f5','nfolio','f12']
pcols = [status_column, upc_column, cost_column, qty_column]

rdup = res.loc[(res.status_nuevo =='f4 de merma - por duplicidad f12 + upc + cantidad')&(res['Comentario GCO']=='Registro duplicado F12 + UPC + Cantidad')]
# sidup = rdup[rdup.duplicated(['f12', 'prd_upc',	'qproducto','f4_nuevo'])]

f4 = pd.read_csv('input/cierres_nc/210901-1556-f4.csv', sep=';', dtype='object')
f4['aa creacion'] = f4['fecha_creacion'].str.split('-').str[2]
f4.cantidad = f4.cantidad.astype('float')

df1 = rdup.copy()
df2 = ica.get_fnan( df1, fcols[1], 'F4')
if df2.empty == False:
    df3 = ica.get_duplicates( df2, [fcols[4], pcols[1], pcols[3], 'f4_nuevo'], 'F12 + UPC + Cantidad + F4')
    ne = ica.get_notfound( df3, f4, [fcols[1], pcols[1]], ['nro_red_inventario','upc'], 'nro_red_inventario', 'F4|UPC|QTY')
    df4 = pd.merge(df3, f4, left_on=[fcols[1], pcols[1]], right_on=['nro_red_inventario','upc'])
    if df4.empty ==False: 
        df5 = ica.get_equalvalue(df4, 'estado', 'anulado', 'ANU', 'Registro anulado')
        df6 = ica.get_diffvalue(df5, 'aa creacion', '2021', 'NAA', f'Registro con año de creación diferente a 2021')
        df7 = ica.get_diffqty_pro(df6, pcols[3], 'cantidad',fcols[3],'nro_red_inventario', 'Cantidad de los F11s de un F4 > cantidad del F4')
        iokf4 = df7[index_name].values
        ica.update_db(iokf4,'GCO', 'OKK')
        ica.update_db(iokf4,'Comentario GCO', 'Coincidencia exacta F4+UPC+QTY')
        ica.get_okk_dup(iokf4, 'Comentario GCO', 'F4+UPC+QTY')
        #ica.get_dup_i(iokf4, 'F4+UPC+QTY')

res = ica.get_db()

In [7]:
res.groupby('GCO')['total_costo_promedio'].sum()

GCO
ANU        9069562
DUP        3930180
NAA       15438360
NCC         239770
NF4          64744
NFD       76389513
OKD      404140798
OKK    10008864569
Name: total_costo_promedio, dtype: int64

## Validicación 1

In [8]:
lista_v1 = [ ['cierre x producto guardado despues de inventario', 'Coincidencia (F12|F11) + Registro duplicado en DB', 'Coincidencia exacta (F12|F11)' ],
       ['cierre x recupero con cliente - refacturacion - base fal.com', 'Coincidencia F12 + Registro duplicado en DB', 'Coincidencia exacta F12'],
       ['cierre x f3 devuelto a proveedor', 'Coincidencia F3+UPC+QTY + Registro duplicado en DB', 'Coincidencia exacta F3+UPC+QTY'],
       ['cierre x producto guardado despues de inventario - no aplica f12', 'Coincidencia (F12|F11) + Registro duplicado en DB', 'Coincidencia exacta (F12|F11)']]

In [9]:
def val_1(bd, status, comment_ant, comment_new):
    dupokk = res.loc[(res['Comentario GCO']==comment_ant) & (res['status_nuevo']==status)]
    f4okk =  res.loc[(res['Comentario GCO']=='Coincidencia F4+UPC+QTY + Registro duplicado en DB') & (res.status_nuevo =='f4 de merma - por duplicidad f12 + upc + cantidad')]
    dupuf4 = f4okk.merge(dupokk, how='inner', on=['f12', 'prd_upc'])
    bd.loc[dupuf4.indice_cf11_x, 'Comentario GCO'] = 'Coincidencia exacta F4+UPC+QTY'
    bd.loc[dupuf4.indice_cf11_x, 'GCO'] = 'OKK'
    bd.loc[dupuf4.indice_cf11_y, 'Comentario GCO'] = comment_new
    bd.loc[dupuf4.indice_cf11_y, 'GCO'] = 'OKK'
    return bd

In [10]:
for i in lista_v1:
    res = val_1(res, i[0], i[1], i[2])

In [11]:
res.groupby('GCO')['total_costo_promedio'].sum()

GCO
ANU        9069562
DUP        3930180
NAA       15438360
NCC         239770
NF4          64744
NFD       76389513
OKD      319956875
OKK    10093048492
Name: total_costo_promedio, dtype: int64

## Validación 2 

In [12]:
lista_v2 = [
    ['cierre x f4 cobrado a terceros', 'Coincidencia F4+UPC+QTY + Registro duplicado en DB', 'Coincidencia exacta F4+UPC+QTY'], 
    ['f4 de merma', 'Coincidencia F4+UPC+QTY + Registro duplicado en DB', 'Coincidencia exacta F4+UPC+QTY']
]

In [13]:
def val_2(bd, status, comment_ant, comment_new):
    dupokk = res.loc[(res['Comentario GCO']==comment_ant) & (res['status_nuevo']==status)]
    f4okk =  res.loc[(res['Comentario GCO']=='Coincidencia F4+UPC+QTY + Registro duplicado en DB') & (res.status_nuevo =='f4 de merma - por duplicidad f12 + upc + cantidad')]
    dupuf4 = f4okk.merge(dupokk, how='inner', on=['f12', 'prd_upc'])
    # Detecta si el registro que está en f4 de merma tiene el mismo # de F4
    nodiff4 = dupuf4.loc[dupuf4.f4_nuevo_x == dupuf4.f4_nuevo_y]
    bd.loc[nodiff4.indice_cf11_x, 'Comentario GCO'] = 'Coincidencia F4+UPC+QTY - Duplicado en DB con igual Nro. de f4'
    bd.loc[nodiff4.indice_cf11_x, 'GCO'] = 'OKD'
    bd.loc[nodiff4.indice_cf11_y, 'Comentario GCO'] = 'Coincidencia F4+UPC+QTY - Duplicado en DB con igual Nro. de f4'
    bd.loc[nodiff4.indice_cf11_y, 'GCO'] = 'OKD'

    dupuf4 = dupuf4.loc[dupuf4.f4_nuevo_x != dupuf4.f4_nuevo_y]
    bd.loc[dupuf4.indice_cf11_x, 'Comentario GCO'] = 'Coincidencia exacta F4+UPC+QTY'
    bd.loc[dupuf4.indice_cf11_x, 'GCO'] = 'OKK'
    bd.loc[dupuf4.indice_cf11_y, 'Comentario GCO'] = comment_new
    bd.loc[dupuf4.indice_cf11_y, 'GCO'] = 'OKK'
    return bd

In [14]:
for i in lista_v2:
    res = val_2(res, i[0], i[1], i[2])

In [15]:
res.groupby('GCO')['total_costo_promedio'].sum()

GCO
ANU        9069562
DUP        3930180
NAA       15438360
NCC         239770
NF4          64744
NFD       76389513
OKD       16940046
OKK    10396065321
Name: total_costo_promedio, dtype: int64

## Validación 3

In [16]:
lista_v3 = [ ['cierre x producto guardado despues de inventario', 'Coincidencia (F12|F11) + Registro duplicado en DB', 'Coincidencia exacta (F12|F11)' ],
       ['cierre x recupero con cliente - refacturacion - base fal.com', 'Coincidencia F12 + Registro duplicado en DB', 'Coincidencia exacta F12'],
       ['cierre x f3 devuelto a proveedor', 'Coincidencia F3+UPC+QTY + Registro duplicado en DB', 'Coincidencia exacta F3+UPC+QTY'],
       ['cierre x producto guardado despues de inventario - no aplica f12', 'Coincidencia (F12|F11) + Registro duplicado en DB', 'Coincidencia exacta (F12|F11)']]

In [17]:
def val_3(bd, status, comment_ant, comment_new):
    dupokk = res.loc[(res['Comentario GCO']==comment_ant) & (res['status_nuevo']==status)]
    f4okk =  res.loc[(res['Comentario GCO']=='Coincidencia F4+UPC+QTY + Registro duplicado en DB') & (res.status_nuevo =='f4 de merma')]
    dupuf4 = f4okk.merge(dupokk, how='inner', on=['f12', 'prd_upc'])
    bd.loc[dupuf4.indice_cf11_x, 'Comentario GCO'] = 'Coincidencia F4+UPC+QTY - F4 de merma por duplicidad'
    bd.loc[dupuf4.indice_cf11_x, 'GCO'] = 'OKK'
    bd.loc[dupuf4.indice_cf11_y, 'Comentario GCO'] = comment_new
    bd.loc[dupuf4.indice_cf11_y, 'GCO'] = 'OKK'
    return bd

In [18]:
for i in lista_v3:
    res = val_3(res, i[0], i[1], i[2])

In [19]:
res.groupby('GCO')['total_costo_promedio'].sum()

GCO
ANU        9069562
DUP        3930180
NAA       15438360
NCC         239770
NF4          64744
NFD       76389513
OKD       16940046
OKK    10396065321
Name: total_costo_promedio, dtype: int64

In [20]:
md = res.loc[res.status_nuevo=='f4 de merma - por duplicidad f12 + upc + cantidad']
okd = md.loc[md['Comentario GCO']=='Coincidencia F4+UPC+QTY + Registro duplicado en DB']

ec = okd.loc[okd.duplicated(['f12', 'prd_upc', 'qproducto'], keep=False)]
res.loc[ec.indice_cf11, 'Comentario GCO'] = 'Coincidencia F4+UPC+QTY + Error clasificación: F4 merma por duplicidad'
res.loc[ec.indice_cf11, 'GCO'] = 'ECL'

In [24]:
res.to_excel('output/210916-1619-cf11.xlsx',index=False)

In [121]:
ec.head()

,indice_cf11,nfolio,f12,prd_upc,qproducto,xobservacion,total_costo_promedio,estado_actual,status_nuevo,transportadora_nuevo,f3nuevo,f4_nuevo,reporte_a_contabilidad,movimiento_contable,nc,tranf_electro_factura,pv,f5,gco_dup,GCO,Comentario GCO,dupmc,checked,error_ru,gco_dupall,fnandup
9,9,1110437496,12625415565,8806098670444,1.00,12625415565 facturacionse cierra f 111043749...,1153880,entrega total,f4 de merma - por duplicidad f12 + upc + cantidad,NaN,NaN,46300577,NaN,NaN,NaN,NaN,NaN,NaN,y,OKD,Coincidencia F4+UPC+QTY + Registro duplicado e...,NaN,y,NaN,y,NaN
10,10,1110439025,12625415565,8806098670444,1.00,12625415565 facturacioncierre por merma de f 1...,1153880,entrega total,f4 de merma - por duplicidad f12 + upc + cantidad,NaN,NaN,46277633,NaN,NaN,NaN,NaN,NaN,NaN,y,OKD,Coincidencia F4+UPC+QTY + Registro duplicado e...,NaN,y,NaN,y,NaN
204,204,1110095283,12602921317,2006939685002,1.00,12602921317 facturacioncierre por merma de f 1...,33056,entrega total,f4 de merma - por duplicidad f12 + upc + cantidad,NaN,NaN,46274632,NaN,NaN,NaN,NaN,NaN,NaN,y,OKD,Coincidencia F4+UPC+QTY + Registro duplicado e...,NaN,y,NaN,y,NaN
205,205,1110212673,12602921317,2006939685002,1.00,12602921317 refacturacionse cierra f 111021267...,33056,entrega total,f4 de merma - por duplicidad f12 + upc + cantidad,NaN,NaN,46298249,NaN,NaN,NaN,NaN,NaN,NaN,y,OKD,Coincidencia F4+UPC+QTY + Registro duplicado e...,NaN,y,NaN,y,NaN
233,233,1110122972,12601700357,6970280948115,1.00,12601700357 refacturacionse cierra f 11101229...,66554,entrega total,f4 de merma - por duplicidad f12 + upc + cantidad,NaN,NaN,46206565,NaN,NaN,NaN,NaN,NaN,NaN,y,OKD,Coincidencia F4+UPC+QTY + Registro duplicado e...,NaN,y,NaN,y,NaN


In [122]:
res.loc[(res.f12 =='12602921317') & (res.prd_upc =='2006939685002')]

,indice_cf11,nfolio,f12,prd_upc,qproducto,xobservacion,total_costo_promedio,estado_actual,status_nuevo,transportadora_nuevo,f3nuevo,f4_nuevo,reporte_a_contabilidad,movimiento_contable,nc,tranf_electro_factura,pv,f5,gco_dup,GCO,Comentario GCO,dupmc,checked,error_ru,gco_dupall,fnandup
204,204,1110095283,12602921317,2006939685002,1.00,12602921317 facturacioncierre por merma de f 1...,33056,entrega total,f4 de merma - por duplicidad f12 + upc + cantidad,NaN,NaN,46274632,NaN,NaN,NaN,NaN,NaN,NaN,y,ECL,Coincidencia F4+UPC+QTY + Error clasificación:...,NaN,y,NaN,y,NaN
205,205,1110212673,12602921317,2006939685002,1.00,12602921317 refacturacionse cierra f 111021267...,33056,entrega total,f4 de merma - por duplicidad f12 + upc + cantidad,NaN,NaN,46298249,NaN,NaN,NaN,NaN,NaN,NaN,y,ECL,Coincidencia F4+UPC+QTY + Error clasificación:...,NaN,y,NaN,y,NaN


In [128]:
aux = ec.merge(res, how='inner', on=['f12', 'prd_upc'])

In [132]:
aux.loc[aux.status_nuevo_x != aux.status_nuevo_y]

,indice_cf11_x,nfolio_x,f12,prd_upc,qproducto_x,xobservacion_x,total_costo_promedio_x,estado_actual_x,status_nuevo_x,transportadora_nuevo_x,f3nuevo_x,f4_nuevo_x,reporte_a_contabilidad_x,movimiento_contable_x,nc_x,tranf_electro_factura_x,pv_x,f5_x,gco_dup_x,GCO_x,Comentario GCO_x,dupmc_x,checked_x,error_ru_x,gco_dupall_x,fnandup_x,indice_cf11_y,nfolio_y,qproducto_y,xobservacion_y,total_costo_promedio_y,estado_actual_y,status_nuevo_y,transportadora_nuevo_y,f3nuevo_y,f4_nuevo_y,reporte_a_contabilidad_y,movimiento_contable_y,nc_y,tranf_electro_factura_y,pv_y,f5_y,gco_dup_y,GCO_y,Comentario GCO_y,dupmc_y,checked_y,error_ru_y,gco_dupall_y,fnandup_y
704,13923,119952023,12582071194,2003419727009,1.00,12582071194 nota credito 12582071194 46055262,35685,entrega total,f4 de merma - por duplicidad f12 + upc + cantidad,NaN,NaN,46467164,NaN,NaN,NaN,NaN,NaN,NaN,y,OKD,Coincidencia F4+UPC+QTY + Registro duplicado e...,NaN,y,NaN,y,NaN,11433,119715499,1.00,12582071194 facturacion se cierra f11 asocia...,35685,entrega total,cierre x f4 cobrado a terceros,deprisa,NaN,46055262,con informacion contable,pv factura,NaN,NaN,pv1001197429,NaN,y,NAA,Registro con año de creación diferente a 2021,NaN,y,NaN,y,NaN
707,16065,1110104626,12582071194,2003419727009,1.00,12582071194 facturacion 12582071194 46055262,35685,entrega total,f4 de merma - por duplicidad f12 + upc + cantidad,NaN,NaN,46467195,NaN,NaN,NaN,NaN,NaN,NaN,y,OKD,Coincidencia F4+UPC+QTY + Registro duplicado e...,NaN,y,NaN,y,NaN,11433,119715499,1.00,12582071194 facturacion se cierra f11 asocia...,35685,entrega total,cierre x f4 cobrado a terceros,deprisa,NaN,46055262,con informacion contable,pv factura,NaN,NaN,pv1001197429,NaN,y,NAA,Registro con año de creación diferente a 2021,NaN,y,NaN,y,NaN
922,20439,1110164583,12,2998815746276,2.00,ingreso de f11 entrega a vas,38076,entrega total,f4 de merma - por duplicidad f12 + upc + cantidad,NaN,NaN,46439840,NaN,NaN,NaN,NaN,NaN,NaN,y,OKD,Coincidencia F4+UPC+QTY + Registro duplicado e...,NaN,y,NaN,y,NaN,1624,119832417,1.00,se cierra f11 se genera f4 46118998 a mavesa p...,19038,entrega total,f4 de merma,NaN,NaN,46118998,NaN,NaN,NaN,NaN,NaN,NaN,n,OKK,Coincidencia exacta F4+UPC+QTY,NaN,y,NaN,NaN,NaN
925,22162,1110478833,12,2998815746276,2.00,f11 entregado al area de vas autoriza aaron,38076,entrega total,f4 de merma - por duplicidad f12 + upc + cantidad,NaN,NaN,46439857,NaN,NaN,NaN,NaN,NaN,NaN,y,OKD,Coincidencia F4+UPC+QTY + Registro duplicado e...,NaN,y,NaN,y,NaN,1624,119832417,1.00,se cierra f11 se genera f4 46118998 a mavesa p...,19038,entrega total,f4 de merma,NaN,NaN,46118998,NaN,NaN,NaN,NaN,NaN,NaN,n,OKK,Coincidencia exacta F4+UPC+QTY,NaN,y,NaN,NaN,NaN
928,20440,1110164583,12,2998815746313,3.00,ingreso de f11 entrega a vas,56915,entrega total,f4 de merma - por duplicidad f12 + upc + cantidad,NaN,NaN,46442345,NaN,NaN,NaN,NaN,NaN,NaN,y,OKD,Coincidencia F4+UPC+QTY + Registro duplicado e...,NaN,y,NaN,y,NaN,1623,119832417,1.00,se cierra f11 se genera f4 46118998 a mavesa p...,18972,entrega total,f4 de merma,NaN,NaN,46118998,NaN,NaN,NaN,NaN,NaN,NaN,n,OKK,Coincidencia exacta F4+UPC+QTY,NaN,y,NaN,NaN,NaN
931,22163,1110478833,12,2998815746313,3.00,f11 entregado al area de vas autoriza aaron,56915,entrega total,f4 de merma - por duplicidad f12 + upc + cantidad,NaN,NaN,46441898,NaN,NaN,NaN,NaN,NaN,NaN,y,OKD,Coincidencia F4+UPC+QTY + Registro duplicado e...,NaN,y,NaN,y,NaN,1623,119832417,1.00,se cierra f11 se genera f4 46118998 a mavesa p...,18972,entrega total,f4 de merma,NaN,NaN,46118998,NaN,NaN,NaN,NaN,NaN,NaN,n,OKK,Coincidencia exacta F4+UPC+QTY,NaN,y,NaN,NaN,NaN


In [133]:
res.loc[res.f12 =='12']

,indice_cf11,nfolio,f12,prd_upc,qproducto,xobservacion,total_costo_promedio,estado_actual,status_nuevo,transportadora_nuevo,f3nuevo,f4_nuevo,reporte_a_contabilidad,movimiento_contable,nc,tranf_electro_factura,pv,f5,gco_dup,GCO,Comentario GCO,dupmc,checked,error_ru,gco_dupall,fnandup
1596,1596,119919101,12,639089,2.00,prestamo se cierra con f4 46138415,22372,entrega total,f4 de merma,NaN,NaN,46138415,NaN,NaN,NaN,NaN,NaN,NaN,n,OKK,Coincidencia exacta F4+UPC+QTY,NaN,y,NaN,NaN,NaN
1597,1597,119892336,12,557733,2.00,se cierra con f4 46138439,14738,entrega total,f4 de merma,NaN,NaN,46138439,NaN,NaN,NaN,NaN,NaN,NaN,n,OKK,Coincidencia exacta F4+UPC+QTY,NaN,y,NaN,NaN,NaN
1598,1598,119892336,12,557059,2.00,se cierra con f4 46138439,22515,entrega total,f4 de merma,NaN,NaN,46138439,NaN,NaN,NaN,NaN,NaN,NaN,n,OKK,Coincidencia exacta F4+UPC+QTY,NaN,y,NaN,NaN,NaN
1599,1599,119832417,12,2998818584714,1.00,se cierra f11 se genera f4 46118998 a mavesa p...,26242,entrega total,f4 de merma,NaN,NaN,46118998,NaN,NaN,NaN,NaN,NaN,NaN,n,OKK,Coincidencia exacta F4+UPC+QTY,NaN,y,NaN,NaN,NaN
1600,1600,119832417,12,2998818584639,1.00,se cierra f11 se genera f4 46118998 a mavesa p...,26250,entrega total,f4 de merma,NaN,NaN,46118998,NaN,NaN,NaN,NaN,NaN,NaN,n,OKK,Coincidencia exacta F4+UPC+QTY,NaN,y,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27025,27025,1110046414,12,2998817724876,1.00,0,16043,despachado,f4 de merma,NaN,NaN,46451453,NaN,NaN,NaN,NaN,NaN,NaN,n,OKK,Coincidencia exacta F4+UPC+QTY,NaN,y,NaN,NaN,NaN
27036,27036,1110052675,12,7703885090014,1.00,0,12630,despachado,f4 de merma,NaN,NaN,46451460,NaN,NaN,NaN,NaN,NaN,NaN,n,OKK,Coincidencia exacta F4+UPC+QTY,NaN,y,NaN,NaN,NaN
27057,27057,1110052675,12,2998817105163,1.00,0,9000,despachado,f4 de merma,NaN,NaN,46451477,NaN,NaN,NaN,NaN,NaN,NaN,n,OKK,Coincidencia exacta F4+UPC+QTY,NaN,y,NaN,NaN,NaN
27067,27067,1110052675,12,2004185342007,1.00,0,6700,despachado,f4 de merma,NaN,NaN,46451484,NaN,NaN,NaN,NaN,NaN,NaN,n,OKK,Coincidencia exacta F4+UPC+QTY,NaN,y,NaN,NaN,NaN


## Validación cf11 2021


In [18]:
f = pd.read_excel('output/cierres_f11/cd/210906-1136-cf11_cd_21-output.xlsx', sheet_name='210906-1136_cf11_cd_21', dtype='object')

In [29]:
inter = f.loc[f.status_final!='error en cierre f11']

In [30]:
f4okk =  f.loc[(f['Comentario GCO']=='Registro sin nro. de F4') & (f.status_final =='error en cierre f11')]
dupuf4 = f4okk.merge(inter, how='inner', on=[f'f12', 'prd_upc']) # por cantidad
# lo que no cruza 

In [31]:
nodiff4 = dupuf4.loc[dupuf4.nfolio_x == dupuf4.nfolio_y]
f.loc[nodiff4.indice_cf11_x, 'Comentario GCO'] = 'Duplicado en DB con igual Nro. de f11'
f.loc[nodiff4.indice_cf11_x, 'GCO'] = 'OKD'

dupuf4 = dupuf4.loc[dupuf4.nfolio_x != dupuf4.nfolio_y]
f.loc[dupuf4.indice_cf11_x, 'Comentario GCO'] = 'Coincidencia exacta F12+UPC+QTY con diferente nro. F11'
f.loc[dupuf4.indice_cf11_x, 'GCO'] = 'OKK'

In [32]:
f.to_excel('output/210906-1234-cf11_21.xlsx',index=False)

F11 tiendas

In [33]:
t = pd.read_excel('input/bases/210903_cf11_tienda_20_base.xlsx', sheet_name='Base', dtype='object')
a = pd.read_excel('input/bases/210906_ajustes.xlsx', sheet_name='F11', dtype='object')

In [34]:
a.columns

Index(['NFOLIO', 'NSECUENCIA', 'PRD_UPC', 'XSUBPROD', 'QPRODUCTO',
       'MPRECIO_VTA', 'TOTAL_MPRECIO', 'CESTADO_DT', 'DPACTADA_DT',
       'CSUCURSAL',
       ...
       'ESTADO ACTUAL DEL F11 - 9 AGO', 'ESTADO 9 AGO',
       'ESTADO ACTUAL DEL F11 - 11AGO', 'ESTADO 11 AGO',
       'ESTADO ACTUAL DEL F11 - 23 AGO', 'ESTADO 23 AGO',
       'ESTADO ACTUAL DEL F11 - 1SEPT', 'ESTADO 1 SEPT',
       'ESTADO ACTUAL DEL F11 - 6SEPT', 'ESTADO 6SEPT'],
      dtype='object', length=136)

In [35]:
t.columns

Index(['NFOLIO', 'NSECUENCIA', 'PRD_UPC', 'XSUBPROD', 'QPRODUCTO',
       'MPRECIO_VTA', 'TOTAL_MPRECIO', 'CESTADO_DT', 'DPACTADA_DT',
       'CSUCURSAL', 'CRUT_CLT', 'XDV_RUT', 'PROPIETARIO', 'DPACTADA_HD',
       'XNOMBRE_CLT', 'XAPELLIDO_CLT', 'XDIREC_CLT', 'XCIUDAD_CLT', 'CCOMUNA',
       'DCREACION', 'DDESPACHO', 'QIMPRESO', 'XOBSERVACION', 'CESTADO',
       'XESTADO', 'CSERVICIO', 'XSERVICIO', 'BORIGEN', 'SUC', 'NAME_ORG',
       'PRD_LVL_CHILD', 'COSTO_PROMEDIO', 'TOTAL_COSTO_PROMEDIO', 'DIAS',
       'ANTIGÜEDAD', 'GRUPO', 'Fecha Base Local', 'Estado 22 abril',
       'Estado hoy', 'MOTIVO', 'F', 'Estado Conciliación ', 'Unnamed: 42'],
      dtype='object')

In [36]:
tua = t.merge(a, how='left', on='NFOLIO')

In [51]:
tua['ESTADO ACTUAL DEL F11 - 6SEPT'].str.lower()

0       entrega total       
1       entrega total       
2       entrega total       
3       entrega total       
4       entrega total       
                ...         
8609    entrega total       
8610    entrega total       
8611    entrega total       
8612    entrega total       
8613    entrega total       
Name: ESTADO ACTUAL DEL F11 - 6SEPT, Length: 8614, dtype: object

In [55]:
t['estado_f11'] = tua['ESTADO ACTUAL DEL F11 - 6SEPT'].str.lower().str.strip()


In [56]:
t.head()

,NFOLIO,NSECUENCIA,PRD_UPC,XSUBPROD,QPRODUCTO,MPRECIO_VTA,TOTAL_MPRECIO,CESTADO_DT,DPACTADA_DT,CSUCURSAL,CRUT_CLT,XDV_RUT,PROPIETARIO,DPACTADA_HD,XNOMBRE_CLT,XAPELLIDO_CLT,XDIREC_CLT,XCIUDAD_CLT,CCOMUNA,DCREACION,DDESPACHO,QIMPRESO,XOBSERVACION,CESTADO,XESTADO,CSERVICIO,XSERVICIO,BORIGEN,SUC,NAME_ORG,PRD_LVL_CHILD,COSTO_PROMEDIO,TOTAL_COSTO_PROMEDIO,DIAS,ANTIGÜEDAD,GRUPO,Fecha Base Local,Estado 22 abril,Estado hoy,MOTIVO,F,Estado Conciliación,Unnamed: 42,estado,estado_f11
0,1110391040,1,8806090523496,GALAXY A21S 64 GB NEGRO,1,779900,779900,1,2021-02-15 00:00:00,SAMSUNG,35,NaN,EMPRESA,2021-02-15 00:00:00,FALABELLA,WTC,AV 6 NORTE # 30-35,CALI,3,2021-01-04 00:00:00,NaN,0,CELULAR NO CARGA INGRESA EN PERFECTO ESTADO N...,1,Ingresado,S,S.TECNICO,F,35,WTC CALI,7929042,"633,646.61","633,646.61",2,1.Menos de 60 días,TIENDAS,2021-01-05 00:00:00,Entrega total,Entrega total,SIN INFORMACIÓN - TIENDA,NaN,CERRADO,NaN,Entrega total,entrega total
1,1110391071,1,6941059635529,REDMI NOTE 8 128GB BLUE,1,1399900,1399900,1,2021-02-15 00:00:00,MN FOTOS,35,NaN,EMPRESA,2021-02-15 00:00:00,FALABELLA,WTC,AV 6 NORTE # 30-35,CALI,3,2021-01-04 00:00:00,NaN,0,PRODUCTO INGRESA EN BUEN ESTADO LA PANTALLA NO...,1,Ingresado,S,S.TECNICO,F,35,WTC CALI,5768665,"731,019.32","731,019.32",2,1.Menos de 60 días,TIENDAS,2021-01-05 00:00:00,Entrega total,Entrega total,SIN INFORMACIÓN - TIENDA,NaN,CERRADO,NaN,Entrega total,entrega total
2,1110391088,1,7709577513335,LED 50 CAIXUN UHD CX50S1USM,1,2199900,2199900,1,2021-02-15 00:00:00,CESER SAS,35,NaN,EMPRESA,2021-02-15 00:00:00,FALABELLA,WTC,AV 6 NORTE # 30-35,CALI,3,2021-01-04 00:00:00,NaN,0,"PUNTO EN PANTALLA, NO SE CONECTA A INTERNET",1,Ingresado,S,S.TECNICO,F,35,WTC CALI,10083468,"976,060.71","976,060.71",2,1.Menos de 60 días,TIENDAS,2021-01-05 00:00:00,Entrega total,Entrega total,F3,32884210,CERRADO,NaN,Entrega total,entrega total
3,1110391101,1,840023204012,MOTOROLA FUSION PLUS BLANCO,1,1699900,1699900,1,2021-02-15 00:00:00,COOPER GLOBAL SAS.,35,NaN,EMPRESA,2021-02-15 00:00:00,FALABELLA,WTC,AV 6 NORTE # 30-35,CALI,3,2021-01-04 00:00:00,NaN,0,PRODUCTO INGRESA EN PERFECTO ESTADO SIN GOLPES...,1,Ingresado,S,S.TECNICO,F,35,WTC CALI,8369457,"804,933.35","804,933.35",2,1.Menos de 60 días,TIENDAS,2021-01-05 00:00:00,Entrega total,Entrega total,SIN INFORMACIÓN - TIENDA,NaN,CERRADO,NaN,Entrega total,entrega total
4,1110391118,1,8806090751967,GALAXY S20 FE CLOUD MINT 128GB,1,2699900,2699900,1,2021-02-15 00:00:00,SAMSUNG,35,NaN,EMPRESA,2021-02-15 00:00:00,FALABELLA,WTC,AV 6 NORTE # 30-35,CALI,3,2021-01-04 00:00:00,NaN,0,"PRODUCTO INGRESA CON RAYONES EN LA PANTALLA , ...",1,Ingresado,S,S.TECNICO,F,35,WTC CALI,11533936,"1,973,876.47","1,973,876.47",2,1.Menos de 60 días,TIENDAS,2021-01-05 00:00:00,Entrega total,Entrega total,SIN INFORMACIÓN - TIENDA,NaN,CERRADO,NaN,Entrega total,entrega total


In [57]:
t.to_csv('output/bases/210906_cf11_20_tienda.csv', sep=';', index=False)